In [1]:
!pip install yfinance timesfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 558.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 4.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of googleapis-common-protos to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [240]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=58cdcab7e3b4593e4a1ba31d0d06f6fba5175d1ddeaa0c9d638723a97a59cf72
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [224]:
import os
import time
from collections import defaultdict
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import timesfm

os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
os.environ['JAX_PMAP_USE_TENSORSTORE'] = 'false'

In [225]:
# Load TimesFM model
timesfm_backend = "gpu"  # or 'cpu'
from jax._src import config
config.update("jax_platforms", {"cpu": "cpu", "gpu": "cuda", "tpu": ""}[timesfm_backend])

model = timesfm.TimesFm(
    context_len=32,
    horizon_len=1,
    input_patch_len=32,
    output_patch_len=128,
    num_layers=20,
    model_dims=1280,
    backend=timesfm_backend,
)
model.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")


Multiprocessing context has already been set.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Constructing model weights.


Constructed model weights in 4.58 seconds.
Restoring checkpoint from /root/.cache/huggingface/hub/models--google--timesfm-1.0-200m/snapshots/8775f7531211ac864b739fe776b0b255c277e2be/checkpoints.


ERROR:absl:For checkpoint version > 1.0, we require users to provide
          `train_state_unpadded_shape_dtype_struct` during checkpoint
          saving/restoring, to avoid potential silent bugs when loading
          checkpoints to incompatible unpadded shapes of TrainState.


Restored checkpoint in 4.76 seconds.
Jitting decoding.
Jitted decoding in 20.59 seconds.


In [242]:
import ta
# Download stock data from Yahoo Finance
ticker = 'AAPL'  # Example: Apple Inc.
data = yf.download(ticker, start="2023-01-01", end="2024-01-01")

# Calculate moving averages
data['moving_avg_10'] = data['Close'].rolling(window=10).mean()
data['moving_avg_50'] = data['Close'].rolling(window=50).mean()
data['rsi'] = ta.momentum.RSIIndicator(data['Close'], window=14).rsi()
data['ema'] = ta.trend.EMAIndicator(data['Close'], window=10).ema_indicator()

# Drop rows with NaN values (caused by rolling window calculations)
data = data.dropna()

df = data[['Close', 'Volume', 'moving_avg_10', 'moving_avg_50', 'rsi', 'ema']].reset_index()
df.columns = ['date', 'close', 'volume', 'moving_avg_10', 'moving_avg_50', 'rsi', 'ema']
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df

[*********************100%%**********************]  1 of 1 completed


,close,volume,moving_avg_10,moving_avg_50,rsi,ema
date,,,,,,
2023-03-15,152.990005,77167900,151.038000,144.793400,58.315418,151.047700
2023-03-16,155.850006,76161100,152.032001,145.409000,62.786074,151.920847
2023-03-17,155.000000,98944600,152.429001,145.981800,60.702342,152.480693
2023-03-20,157.399994,73641400,152.786000,146.629400,64.304515,153.375111
2023-03-21,159.279999,73938300,153.553999,147.222600,66.866621,154.448727
...,...,...,...,...,...,...
2023-12-22,193.600006,37122800,195.747002,185.398801,54.609714,194.957922
2023-12-26,193.050003,28919300,195.734003,185.682801,53.026884,194.611027
2023-12-27,193.149994,48087700,195.578001,185.971400,53.291939,194.345385


In [243]:
def get_batched_data_fn(batch_size: int = 32, context_len: int = 32, horizon_len: int = 1):
    examples = defaultdict(list)
    num_examples = 0
    for start in range(0, len(df) - (context_len + horizon_len), horizon_len):
        num_examples += 1
        context_end = start + context_len
        examples["inputs"].append(df["close"][start:context_end].tolist())
        examples["volume"].append(df["volume"][start:context_end].tolist() + [df["volume"].iloc[context_end - 1]])
        examples["moving_avg_10"].append(df["moving_avg_10"][start:context_end].tolist() + [df["moving_avg_10"].iloc[context_end - 1]])
        examples["moving_avg_50"].append(df["moving_avg_50"][start:context_end].tolist() + [df["moving_avg_50"].iloc[context_end - 1]])
        examples["rsi"].append(df["rsi"][start:context_end].tolist() + [df["rsi"].iloc[context_end - 1]])
        examples["ema"].append(df["ema"][start:context_end].tolist() + [df["ema"].iloc[context_end - 1]])
        examples["outputs"].append(df["close"][context_end:context_end + horizon_len].tolist())
        examples["dates"].append(df.index[start:context_end + horizon_len].tolist())  # Add dates to examples

    def data_fn():
        for i in range(1 + (num_examples - 1) // batch_size):
            yield {k: v[(i * batch_size): ((i + 1) * batch_size)] for k, v in examples.items()}

    return data_fn

In [228]:

# Define metrics
def mse(y_pred, y_true):
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    return np.mean(np.square(y_pred - y_true), axis=1, keepdims=True)

def mae(y_pred, y_true):
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    return np.mean(np.abs(y_pred - y_true), axis=1, keepdims=True)

In [315]:
from itertools import combinations
features = ["volume", "moving_avg_10", "moving_avg_50", "rsi", "ema"]
feature_combinations = []
for r in range(1, len(features) + 1):
    feature_combinations.extend(combinations(features, r))

# Benchmark
batch_size = 32
context_len = 32
horizon_len = 1

results_summary = []

for combo in feature_combinations:

    input_data = get_batched_data_fn(batch_size=batch_size, context_len=context_len, horizon_len=horizon_len)
    metrics = defaultdict(list)
    results = []

    for i, example in enumerate(input_data()):
        raw_forecast, _ = model.forecast(
            inputs=example["inputs"], freq=[0] * len(example["inputs"])
        )
        start_time = time.time()


        dynamic_numerical_covariates = {feature: example[feature] for feature in combo}

        # Forecast with covariates
        cov_forecast, ols_forecast = model.forecast_with_covariates(
            inputs=example["inputs"],
            dynamic_numerical_covariates=dynamic_numerical_covariates,
            dynamic_categorical_covariates={},
            static_numerical_covariates={},
            static_categorical_covariates={},
            freq=[0] * len(example["inputs"]),
            ridge=0.0,
            force_on_cpu=False,
            normalize_xreg_target_per_input=True,
        )
        end_time = time.time()


        for j in range(len(example["inputs"])):
            result = {
                'batch': i,
                'input_index': j,
                'train_start_date': example["dates"][j][0],  # Start date of the training data
                'train_end_date': example["dates"][j][-2],  # End date of the training data
                'predict_date': example["dates"][j][-1],  # Prediction date
                'raw_forecast': raw_forecast[j][0],
                'y_actual': example["outputs"][j][0],
                'y_predicted': cov_forecast[j][0],
                'features': str(combo)
            }
            results.append(result)

    results_df = pd.DataFrame(results)

    # Calculate accuracy
    correct_predictions = 0
    total_predictions = 0

    for i in range(1, len(results_df)):
        actual_today = results_df.iloc[i]['y_actual']
        predicted_today = results_df.iloc[i]['y_predicted']
        actual_yesterday_index = i - 1
        actual_yesterday_date = results_df.iloc[actual_yesterday_index]['predict_date']

        if pd.to_datetime(actual_yesterday_date) != pd.to_datetime(results_df.iloc[i]['predict_date']) - pd.DateOffset(days=1):
            continue

        actual_yesterday = results_df.iloc[actual_yesterday_index]['y_actual']
        actual_movement = actual_today - actual_yesterday
        predicted_movement = predicted_today - actual_yesterday

        if (actual_movement > 0 and predicted_movement > 0) or (actual_movement < 0 and predicted_movement < 0):
            correct_predictions += 1

        total_predictions += 1

    accuracy = correct_predictions / total_predictions

    # Calculate profit
    results_df['signal'] = np.where(results_df['y_predicted'].shift(-1) > results_df['y_predicted'], 'Buy', 'Sell')
    initial_investment = 100
    capital = initial_investment
    shares = 0
    profit = 0
    capital_history = []

    for i in range(len(results_df) - 1):
        i = i + 1
        if results_df.loc[i, 'signal'] == 'Buy' and capital > 0:
            shares = capital / results_df.loc[i, 'y_actual']
            capital = 0
        elif results_df.loc[i, 'signal'] == 'Sell' and shares > 0:
            capital = shares * results_df.loc[i, 'y_actual']
            shares = 0
        capital_history.append(capital + shares * results_df.loc[i, 'y_actual'])

    final_capital = capital + shares * results_df.iloc[-1]['y_actual']
    total_profit = final_capital - initial_investment

    results_summary.append({
        'features': combo,
        'mse_model': mean_squared_error(results_df['y_actual'], results_df['y_predicted']),
        'mse_naive': mean_squared_error(results_df['y_actual'], results_df['raw_forecast']),
        'accuracy': accuracy,
        'profit': total_profit,
        'results_df': results_df
    })

# Compare results
for summary in results_summary:
    print(f"Features: {summary['features']}")
    print(f"MSE Model: {summary['mse_model']}")
    print(f"MSE Naive: {summary['mse_naive']}")
    print(f"Accuracy: {summary['accuracy']}")
    print(f"Total Profit: {summary['profit']} USD\n")




Features: ('volume',)
MSE Model: 6.062546310132824
MSE Naive: 5.199384890329037
Accuracy: 0.4580152671755725
Total Profit: 22.00186269856175 USD

Features: ('moving_avg_10',)
MSE Model: 5.220975099411416
MSE Naive: 5.199384890329037
Accuracy: 0.5114503816793893
Total Profit: 12.040812389773265 USD

Features: ('moving_avg_50',)
MSE Model: 5.106899310945577
MSE Naive: 5.199384890329037
Accuracy: 0.5190839694656488
Total Profit: 13.384196334667052 USD

Features: ('rsi',)
MSE Model: 4.707958026230612
MSE Naive: 5.199384890329037
Accuracy: 0.46564885496183206
Total Profit: 11.259490710695275 USD

Features: ('ema',)
MSE Model: 5.199588593350546
MSE Naive: 5.199384890329037
Accuracy: 0.4732824427480916
Total Profit: 14.111235027202397 USD

Features: ('volume', 'moving_avg_10')
MSE Model: 5.5032082163559695
MSE Naive: 5.199384890329037
Accuracy: 0.5343511450381679
Total Profit: 21.001732005029012 USD

Features: ('volume', 'moving_avg_50')
MSE Model: 5.991322665429574
MSE Naive: 5.1993848903290

In [316]:
best_mse_model = min(results_summary, key=lambda x: x['mse_model'])
best_accuracy = max(results_summary, key=lambda x: x['accuracy'])
best_profit = max(results_summary, key=lambda x: x['profit'])

print("\nBest model based on MSE:")
print(f"Features: {best_mse_model['features']}")
print(f"MSE Model: {best_mse_model['mse_model']}")
print(f"Accuracy: {best_mse_model['accuracy']}")
print(f"Total Profit: {best_mse_model['profit']} USD\n")

print("\nBest model based on accuracy:")
print(f"Features: {best_accuracy['features']}")
print(f"MSE Model: {best_accuracy['mse_model']}")
print(f"Accuracy: {best_accuracy['accuracy']}")
print(f"Total Profit: {best_accuracy['profit']} USD\n")

print("\nBest model based on profit:")
print(f"Features: {best_profit['features']}")
print(f"MSE Model: {best_profit['mse_model']}")
print(f"Accuracy: {best_profit['accuracy']}")
print(f"Total Profit: {best_profit['profit']} USD\n")


Best model based on MSE:
Features: ('volume', 'rsi', 'ema')
MSE Model: 4.659188324701972
Accuracy: 0.44274809160305345
Total Profit: 13.248388322840796 USD


Best model based on accuracy:
Features: ('volume', 'moving_avg_10')
MSE Model: 5.5032082163559695
Accuracy: 0.5343511450381679
Total Profit: 21.001732005029012 USD


Best model based on profit:
Features: ('volume',)
MSE Model: 6.062546310132824
Accuracy: 0.4580152671755725
Total Profit: 22.00186269856175 USD



# buy_and_hold_profit

In [310]:
initial_investment = 100
buy_and_hold_final_value = initial_investment * (results_df['y_actual'].iloc[-1] / results_df['y_actual'].iloc[0])
buy_and_hold_profit = buy_and_hold_final_value - initial_investment
buy_and_hold_profit

14.857012900440239